# Latent Semantic Indexing


Latent semantic indexing is an information retrieval technique based on matrix decomposition. 

It overcomes known issues in the word vector model such as:

- synonymy: When we search for a word in the word vector model, we do not find any of its synonyms. In other words, searching for a term such as "laptop" will not yield results containing "notebook". This is a desired property since more often than not, the synonym is a relevant result for our query.

- polysemy: When a word has multiple meanings the word vector model is not able to capture this. This means that all documents related to the word are return while some of them might actually not be very relevant since we are looking for a particular usage/definition of the word.

These problems are all handled by LSI. The idea is that we use matrix decomposition to map terms and documents into a latent space that represents the concept space. In this (usually reduced) concept space, we find that terms which are related to similar concepts are closer together while those that aren't are further apart. Hence we can improve the quality of retrieval.

# Theoretical Background

The idea behind LSI, is to create the term-document matrix for the corpus. This matrix has all the terms in the dictionary as the rows, and all the documents as the columns. 

The matrix is defined as follows:

$A[i,j] = \delta_{term_i, doc_j}$

where the Kronecker delta is one if $term_i$ is in $doc_j$ and zero otherwise. 

Actually, the term document matrix can contain any measure of "how much" the term is contained in the document. In our implementation, we will use TF-IDF. Therefore, $A[i,j]$ will contain the TF-IDF of $term_i$ in $doc_j$.

Given a term document matrix $A \in \Re_{N \times M}$ of rank $k = \min (N,M)$ first step in LSI is to perform SVD and obtain the three matrices $U$, $\Sigma$, and $V$, where

$A = U \Sigma V^T$.

and $U \in \Re_{N\times k}$, $\Sigma \in \Re_{k\times k}$, and $V \in \Re_{M\times k}$.

The matrix $U$ is the term-concept matrix and each column represents how much each term is represented by that concept. The matrix $V$ is the document-concept matrix and each column represents how much each document contains that concept. Finally, the matrix $\Sigma$ represents the concept matrix and it gives the importance of each concept in the corpus. 

Once we have done the SVD, we can reduce the dimensionality of the concept space by truncating the matrix $\Sigma$ and keeping only the $J$ most important concepts. This in turn implies a truncation for U and V as well. 

This technique is common in dimensionality reduction techniques as we often find that the majority of the information is explained/contained in a smaller subspace. This results in more efficiency at the cost of loosing some information quality. 

Finally, to retrieve documents, we can simply map the query to the latent space, and retrieve the documents most similar to it through the use of cosine similarity.

To do this, we use the following transformation which allows us to transform a document (or query) from the full space to the latent space denoted by hat:

1) $\hat d = \Sigma ^ {-1} U^T d$

# Implementation
This implementation uses LSI on a corpus which is based on wikipedia movies and their plot descriptions. There are approximately 35,000 documents in dataset.

The data set can be found at:

https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots

It was downloaded and uploaded to Google Drive so that we can use colab to run the notebook.

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# Set global variables for executing long pieces of code.
#Execute True skips long pieces of code such as corpus preprocessing and timing of the functions
#However, if no model is saved, execute must be set to true initially. Once the model is saved, it can be set to false.
EXECUTE = False
#Set to true only if preprocessed corpus exists. Otherwise set to false.
LOAD = True



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Data set exploration

We begin by exploring the data set. 

According to the kaggle description of the dataset, there are 8 columns:

- Release Year
- Title
- Origin/Ethnicity
- Director
- Cast 
- Genre
- WikiPage
- Plot

In [ ]:
import pandas as pd
PATH = "/content/gdrive/MyDrive/InformationRetrieval/wiki_movie_plots_deduped.csv"
df = pd.read_csv(PATH)
print(f"The length of the data set is {len(df)} documents")
df.head(10)

The length of the data set is 34886 documents


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
5,1903,Alice in Wonderland,American,Cecil Hepworth,May Clark,unknown,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,"Alice follows a large white rabbit down a ""Rab..."
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...
8,1905,The Little Train Robbery,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Train...,The opening scene shows the interior of the ro...
9,1905,The Night Before Christmas,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Night_Before...,Scenes are introduced using lines of the poem....


In [ ]:
def data_loader(PATH, chunksize):
    '''This is a function generator which yields the titles of the movies and the plot descriptions as chunks so that we can 
      be certain that everything can fit in memory. 
      Input: PATH of the data set (must be a csv file)
      Output: Generator which returns two lists
              - titles list
              - plots list 
    '''
    try:
        with pd.read_csv(PATH, usecols=[1,7],chunksize = chunksize) as reader:
            for chunk in reader:
                yield chunk["Title"], chunk["Plot"]
    except:
        raise Exception("Path does not exist")


## Preprocessing
At this point, we preprocess the corpus via the following pipeline:

- Tokenization: tokenizing the documents using spacys tokenizer.
- Normalization: normalizing the words by converting everything to lower case. We also use regular expressions to remove all non character symbols. 
- Stop Word removal: we will use spacy's stop words for the English language
- Lemmatization: we will keep the lemma of each word as opposed to stemming. Lemmatization uses morphological analysis of the sentence to determine the base form of the word. 

After the corpus has been passed through the pipeline, we will obtain a "clean" version of the text with considerably reduced size.

In [ ]:
import spacy
import re

#load spacys english dictionary. 
nlp = spacy.load("en_core_web_sm")

def normalize(text):
    '''This function removes punctuation and symbols and turns the text into lower case.
      Input: a string of text
      Output: a string of text which has been preprocessed as described above
    '''
    no_punctuation = re.sub(r'[^\w^\s^-]','',text)
    no_symbols = re.sub(r'[^a-zA-Z0-9 ]', '', no_punctuation)
    downcase = no_symbols.lower()
    return downcase

def preprocess_corpus(corpus):
    '''This function preprocesses each document of the corpus by first normalizing it and then keeping only the lemma of the word
     if the word is not a stop word.
     Input: a list of strings (documents)
     Outout: a list of strings (documents) which have been preprocessed.
    '''
    clean_text = [] 
    for document in corpus:
        lowered_text = normalize(document)
        clean_text.append([token.lemma_ for token in nlp(lowered_text) if not token.is_stop])
    return clean_text 

def chunk_preprocessor(PATH, chunksize = 5000):
    '''This function preprocesses each of the chunks produced by the data loader by calling auxiliary functions.
     Input: PATH for the data loader
     Output: 
    '''
    clean_plots = []
    titles = []
    i = 0
    for titles_chunk, plots_chunk in data_loader(PATH, chunksize = chunksize):
        clean_plots.extend(preprocess_corpus(plots_chunk))
        titles.extend(titles_chunk)
        print(f"Chunk {i+1} finished.")
        i+=1

    return titles, clean_plots  

## Running the preprocessing pipeline
Now we can run the chunk preprocessor on the data set. The pipeline process is the most computationally demanding and it takes approximately 30 minutes. For this reason, we save the results into a dictionary so that we dont have to repeat this process again.

In [ ]:
if EXECUTE:
    titles, clean_plots = chunk_preprocessor(PATH)

In [ ]:
import pickle
def save_clean_corpus(PATH, filename = "/content/gdrive/MyDrive/InformationRetrieval/cleaned_corpus.pt"):
    '''Saves the clean corpus after it has been cleaned.
    '''
    titles, clean_plots = chunk_preprocessor(PATH)
    saving_dict = {
        "titles" : titles,
        "plot_description": clean_plots 
    }

    with open(filename, "wb") as fp:
      pickle.dump(saving_dict, fp)

    del saving_dict

def load_clean_corpus(filename = "/content/gdrive/MyDrive/InformationRetrieval/cleaned_corpus.pt"):
    '''Loads the clean corpus at the specified path.
        The output is changed from a list of lists to a list of strings which is the format that is used for the vectorizer. 
    '''
    with open(filename, "rb") as fp:
        new_dict = pickle.load(fp)
    return list(new_dict["titles"]), [" ".join(x) for x in new_dict["plot_description"]]


Once the preprocessing step has been run once, we can simply load the clean corpus and work with that. 

In [ ]:
if LOAD:
  titles, plots = load_clean_corpus()

## Building the Term-Document matrix

As mentioned above in the theoretical backgrounds, we need to build the term document matrix where each entry contains a measure of how much the term is contained inside the document. We can use various types of measures, but in this implementation we will use tf-idf as this better captures the importance of each term inside a document. 

To achieve this, we use scikit-learn TfidfVectorizer which automatically does this. 

In the vectorizer, we can specify various things, such as how many ngrams to use, the maximum and minimun document frequency that we should consider, the stop words that should be removed and other various options. However, since we already removed stop words and we preprocessed the corpus, we will keep the options simple and simply limit the document frequency between 1% and 90% since this eliminates words that are too frequent and lacking information, as well as very rare words which occur so rarely that they are of low importance.  

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
from numpy.linalg import norm


vectorizer = TfidfVectorizer(
    max_df = 0.90,
    min_df = 0.01
    )
# notice that the vectorizer automatically builds a document-term matrix. To stay consistent with the notation discussed in class
# we simply transpose this matrix to obtain a term document matrix. 
X = vectorizer.fit_transform(plots).transpose()

print(X.shape)

(2105, 34886)


As we can see, after the entire clean up phase, the dictionary is composed of 2105 terms. 

To get an idea of the dictionary that was built by the tfidf vectorizer, we can use the "get_feature_names_out()" method. This will show the terms that make up the rows of the term document matrix. 

In [ ]:
#printing first 100 terms of the dictionary
print(vectorizer.get_feature_names_out()[:100])

['10' '20' '30' 'abandon' 'abduct' 'ability' 'able' 'aboard' 'absence'
 'abuse' 'abusive' 'accept' 'access' 'accident' 'accidentally' 'accompany'
 'accomplice' 'accord' 'account' 'accuse' 'achieve' 'acknowledge'
 'acquaintance' 'acquire' 'act' 'action' 'activate' 'activity' 'actor'
 'actress' 'actual' 'actually' 'adam' 'add' 'addition' 'address' 'admit'
 'adopt' 'adult' 'advance' 'advantage' 'adventure' 'advice' 'advise'
 'affair' 'affect' 'affection' 'afraid' 'africa' 'afterward' 'age'
 'agency' 'agent' 'ago' 'agree' 'ahead' 'aid' 'aim' 'air' 'aircraft'
 'airport' 'aka' 'alan' 'alarm' 'alcohol' 'alcoholic' 'alert' 'alex' 'ali'
 'alice' 'alien' 'alive' 'allow' 'ally' 'ambition' 'ambush' 'america'
 'american' 'americans' 'ancient' 'angel' 'angeles' 'anger' 'angrily'
 'angry' 'animal' 'ann' 'anna' 'anne' 'announce' 'answer' 'anthony'
 'anymore' 'apart' 'apartment' 'apologize' 'apparent' 'apparently'
 'appeal' 'appear']


Below we can even visualize the term document matrix. As we can see, the values are very sparse, thus it is somewhat useless in terms of visual cues.

In [ ]:
df2 = pd.DataFrame(X.toarray(), index = vectorizer.get_feature_names_out())
df2.columns = titles
df2.head(20)

,Kansas Saloon Smashers,Love by the Light of the Moon,The Martyred Presidents,"Terrible Teddy, the Grizzly King",Jack and the Beanstalk,Alice in Wonderland,The Great Train Robbery,The Suburbanite,The Little Train Robbery,The Night Before Christmas,...,Selam,Particle (film),Mandıra Filozofu,Winter Sleep,Sivas,The Water Diviner,Çalgı Çengi İkimiz,Olanlar Oldu,Non-Transferable,İstanbul Kırmızısı
10,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
20,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
30,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
abandon,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.079304,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
abduct,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
ability,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
able,0.0,0.0,0.0,0.0,0.070668,0.0,0.0,0.0,0.000000,0.0,...,0.032043,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.099393,0.0
aboard,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
absence,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
abuse,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [ ]:
df2_mask = df2.iloc[:,0] > 0
df2[df2_mask]

,Kansas Saloon Smashers,Love by the Light of the Moon,The Martyred Presidents,"Terrible Teddy, the Grizzly King",Jack and the Beanstalk,Alice in Wonderland,The Great Train Robbery,The Suburbanite,The Little Train Robbery,The Night Before Christmas,...,Selam,Particle (film),Mandıra Filozofu,Winter Sleep,Sivas,The Water Diviner,Çalgı Çengi İkimiz,Olanlar Oldu,Non-Transferable,İstanbul Kırmızısı
appear,0.146445,0.000000,0.000000,0.067532,0.0,0.065453,0.000000,0.0,0.033335,0.000000,...,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
assault,0.209841,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.00000,0.000000,0.000000,0.054428,0.0,0.0,0.000000,0.0
bar,0.187628,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
begin,0.111165,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.07775,0.000000,0.000000,0.057667,0.0,0.0,0.000000,0.0
break,0.125124,0.000000,0.000000,0.000000,0.0,0.000000,0.067036,0.0,0.028482,0.000000,...,0.050867,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.078891,0.0
burst,0.237186,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
cash,0.223491,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
customer,0.239231,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
drink,0.180406,0.000000,0.000000,0.000000,0.0,0.080632,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.00000,0.000000,0.000000,0.046793,0.0,0.0,0.000000,0.0
dump,0.229598,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0


In [ ]:
del df2

## SVD
We perform Singular Value Decomposition using the "svds" function from scipy.sparse.linalg. The reason we use sparse methods is because the vectorizer returns a sparse data object which aids in memory handling. 

In [ ]:
from scipy.sparse.linalg import svds
import scipy.sparse as sparse

U, S, Vh = svds(X,  k = 100)

print(U.shape)
print(S.shape)
print(Vh.shape)

(2105, 100)
(100,)
(100, 34886)


## Answering the query

To answer a query, we must first map it into the latent space and then compute cosine similarity with the mapped documents in that space. 
To do this, we simply 

- Preprocess the query using the same pipeline to convert $q \to q_c$ where $q_c$ is the cleaned query.
- Use the same vectorizer to construct a tfidf vector and transform $q_c \to \vec q_c$. Note that by using the same vectorizer, we automatically eliminate any words from the query that were not present in the corpus! 
- Use the relation: $\vec q_{rc} = \Sigma ^ {-1} U^T \vec q_c$ to map the cleaned query vector into the reduce latent space.  

In [ ]:
def map_text_to_reduced_space(text, corpus_vectorizer, U, S):
    '''This function maps text from the full space to the latent space by cleaning it, vectorizing it and then
        using the matrices from SVD.
        Input: text to be processed, the vectorizer of the corpus, U and Sigma matrices
        Output: a vector of dimension (k,) which are the latent dimensions chosen for U and S. It represents the mapping of the 
                text in the latent reduced space.
    '''
    clean_text = preprocess_corpus([text])
    clean_text = [" ".join(x) for x in clean_text]
    vector_text = corpus_vectorizer.transform(clean_text).transpose()
    return 1/S*sparse.csr_matrix.dot(U.T, vector_text).squeeze()

As an example query, we take the full plot description of the first movie in the data set (uncleaned)

In [ ]:
query, title = (df["Plot"][0], df["Title"][0])
%timeit reduced_text = map_text_to_reduced_space(query, vectorizer, U, S)

20.6 ms ± 804 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Now we can answer a query by simply mapping it into the reduced latent space and then computing the cosine similarity with all the documents in the space. 

We sort the documents in reverse order according to the similarity.

In [ ]:
from collections import namedtuple

ranked_doc_info = namedtuple('ranked_doc_info', ["similarity_score", "index", "title"])

def answer_query(query):
    ''' This function answers a query by mapping to the reduced latent space.
        Input: query text, corpus vectorizer, U and Sigma from SVD.
        Output: A list of tuples containing (similarity score, docID, title of movie). The list
                ordered from highest similarity to lowest.
    '''
    mapped_query = map_text_to_reduced_space(query, vectorizer, U, S)
    norm_mq = norm(mapped_query)
    mapped_query /= norm_mq
    ranked_docs = []
    for i, row in enumerate(Vh.T):
        norm_r = norm(row)
        if norm_r < 1e-15:
          continue
        else:
          ranked_docs.append(ranked_doc_info(np.dot(mapped_query, row)/norm_r, i, titles[i]))
    ranked_docs.sort(key = lambda x:x[0], reverse = True)
    return ranked_docs

### Top k Retrieval
Lastly, we perform top k retreival by simply returning the first k documents from the list that is returned by the query answer. 

In [ ]:
def top_k_retrieval(query, k):
    '''This function retrieves the top k documents from the results of the query answer.
    '''
    assert k < len(titles), f"Error: {k} > {len(titles)}."
    return answer_query(query)[:k]

In [ ]:
docs = top_k_retrieval(query, 10)

In [ ]:
print(f"Query title is {title}")
for doc in docs:
    print(doc)

Query title is Kansas Saloon Smashers
ranked_doc_info(similarity_score=1.0, index=0, title='Kansas Saloon Smashers')
ranked_doc_info(similarity_score=0.7788359831734177, index=6364, title='One Froggy Evening')
ranked_doc_info(similarity_score=0.7242336304598825, index=9043, title='Golden Needles')
ranked_doc_info(similarity_score=0.7242336304598825, index=23040, title='Golden Needles')
ranked_doc_info(similarity_score=0.7181273499299475, index=4917, title='Incident')
ranked_doc_info(similarity_score=0.7093090308663197, index=18168, title='Men Like These')
ranked_doc_info(similarity_score=0.6985558194210181, index=1799, title='This Side of Heaven')
ranked_doc_info(similarity_score=0.697046691597584, index=17711, title='The Tracker')
ranked_doc_info(similarity_score=0.6834659983178677, index=2170, title='Arizona Days')
ranked_doc_info(similarity_score=0.6754434904814739, index=18579, title='Facing the Music')


# Putting it all together

We can create a class that performs all these functionalities

In [ ]:
import pandas as pd
import spacy
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
from numpy.linalg import norm
from scipy.sparse.linalg import svds
import scipy.sparse as sparse
from collections import namedtuple


ranked_doc_info = namedtuple('ranked_doc_info', ["similarity_score", "index", "title"])


In [ ]:
def data_loader(PATH, chunksize):
    '''This is a function generator which yields the titles of the movies and the plot descriptions as chunks so that we can 
      be certain that everything can fit in memory. 
      Input: PATH of the data set (must be a csv file)
      Output: Generator which returns two lists
              - titles list
              - plots list 
    '''
    try:
        with pd.read_csv(PATH, usecols=[1,7],chunksize = chunksize) as reader:
            for chunk in reader:
                yield chunk["Title"], chunk["Plot"]
    except:
        raise Exception("Path does not exist")

nlp = spacy.load("en_core_web_sm")

def normalize(text):
    '''This function removes punctuation and symbols and turns the text into lower case.
      Input: a string of text
      Output: a string of text which has been preprocessed as described above
    '''
    no_punctuation = re.sub(r'[^\w^\s^-]','',text)
    no_symbols = re.sub(r'[^a-zA-Z0-9 ]', '', no_punctuation)
    downcase = no_symbols.lower()
    return downcase

def preprocess_corpus(corpus):
    '''This function preprocesses each document of the corpus by first normalizing it and then keeping only the lemma of the word
     if the word is not a stop word.
     Input: a list of strings (documents)
     Outout: a list of strings (documents) which have been preprocessed.
    '''
    clean_text = [] 
    for document in corpus:
        lowered_text = normalize(document)
        clean_text.append([token.lemma_ for token in nlp(lowered_text) if not token.is_stop])
    return clean_text 

def chunk_preprocessor(PATH, chunksize = 5000):
    '''This function preprocesses each of the chunks produced by the data loader by calling auxiliary functions.
     Input: PATH for the data loader
     Output: 
    '''
    clean_plots = []
    titles = []
    i = 0
    for titles_chunk, plots_chunk in data_loader(PATH, chunksize = chunksize):
        clean_plots.extend(preprocess_corpus(plots_chunk))
        titles.extend(titles_chunk)
        print(f"Chunk {i+1} finished.")
        i+=1

    return titles, clean_plots  

In [ ]:
class LSI():
    def __init__(self, components, load_path =  "/content/gdrive/MyDrive/InformationRetrieval/cleaned_corpus.pt", path_to_raw_data = None):
        if path_to_raw_data is not None:
            self.path_to_raw_data =  path_to_raw_data
            self.clean_and_save_corpus(data_path = self.path_to_raw_data, filename = self.load_path)

        self.titles, self.corpus = self.load_clean_corpus(load_path)
        self.raw_corpus = pd.read_csv("/content/gdrive/MyDrive/InformationRetrieval/wiki_movie_plots_deduped.csv", usecols = [7])
        self.raw_corpus.index = self.titles
        self.vectorizer = TfidfVectorizer(
            max_df = 0.90,
            min_df = 0.01
            )   
        self.term_document_matrix = self.tfidf_vectorizer()
        self.U, self.S, self.Vh = svds(self.term_document_matrix,components)

    @classmethod
    def load_clean_corpus(cls,filename):
        '''Loads the clean corpus at the specified path.
        The output is changed from a list of lists to a list of strings which is the format that is used for the vectorizer. 
        '''
        with open(filename, "rb") as fp:
            new_dict = pickle.load(fp)
        return list(new_dict["titles"]), [" ".join(x) for x in new_dict["plot_description"]]
    @classmethod
    def clean_and_save_corpus(cls,data_path, filename):
        '''Saves the clean corpus after it has been cleaned.
        '''
        titles, clean_plots = chunk_preprocessor(data_path)
        saving_dict = {
            "titles" : titles,
            "plot_description": clean_plots 
        }

        with open(filename, "wb") as fp:
          pickle.dump(saving_dict, fp)

        del saving_dict
    def tfidf_vectorizer(self):
        '''Creates the term document matrix from the clean corpus.
        '''
        X = self.vectorizer.fit_transform(self.corpus).transpose()
        return X
    def svds(self, components):
        '''Performs truncated svd on the term document matrix by specifying the number of components.
        '''
        U, S, Vh = svds(self.term_document_matrix,  k = components, random_state = 1234)
        return U,S,Vh
    def change_number_of_components(self, components):
        '''If number of components must be changed, we only need to perform svds again. This does this and avoids reconstructing the term document
           matrix.
        '''
        self.U, self.S, self.Vh = svds(self.term_document_matrix, components)
    def clean_and_vectorize_text(self,text):
        '''Given a text, preprocesses it and uses the vectorizer for the corpus to construct the tfidf vector.
        '''
        clean_text = preprocess_corpus([text])
        clean_text = [" ".join(x) for x in clean_text]
        vector_text = self.vectorizer.transform(clean_text).transpose()
        return vector_text
    def map_text_to_reduced_space(self,text):
        '''This function maps text from the full space to the latent space using the matrices from SVD.
            Input: string of text
            Output: a vector of dimension (k,) which are the latent dimensions chosen for U and S. It represents the mapping of the 
                    text in the latent reduced space.
        '''
        vector_text = self.clean_and_vectorize_text(text)
        return 1/self.S*sparse.csr_matrix.dot(self.U.T, vector_text).squeeze()
    def answer_query(self,query):
        ''' This function answers a query by mapping to the reduced latent space.
            Input: query text (string)
            Output: A list of named tuples containing (similarity score, docID, title of movie). The list is
                 ordered from highest similarity to lowest.
        '''
        mapped_query = self.map_text_to_reduced_space(query)
        mapped_query /= norm(mapped_query)
        ranked_docs = []
        for i, row in enumerate(self.Vh.T):
            norm_r = norm(row)
            if norm_r < 1e-15:
              continue
            else:
              ranked_docs.append(ranked_doc_info(np.dot(mapped_query, row)/norm_r, i, self.titles[i]))
        ranked_docs.sort(key = lambda x:x[0], reverse = True)
        return ranked_docs
    def top_k_retrieval(self, query, k, print_info = True):
        '''This function retrieves the top k documents from the results of the query answer.
        '''
        assert k < len(self.titles), f"Error: {k} > {len(self.titles)}."
        query_answer = self.answer_query(query)[:k]
        if print_info:
            for top_docs in query_answer:
                print(top_docs)
        return query_answer
   
    def get_from_corpus(self, *args):
        assert len(args) == 1, "only one argument possible"
        if isinstance(args[0], str):
            return self.raw_corpus["Plot"].loc[args[0]]
        elif isinstance(args[0], int):
            return self.raw_corpus["Plot"].iloc[args[0]]
        else:
            raise Exception("Invalid type: argument can only be an int or a string")
      



In [ ]:
movie_LSI = LSI(components=100)
query, title = (df["Plot"][0], df["Title"][0])
print(f"Query title is {title}")
docs = movie_LSI.top_k_retrieval(query, 10)

Query title is Kansas Saloon Smashers
ranked_doc_info(similarity_score=1.0000000000000002, index=0, title='Kansas Saloon Smashers')
ranked_doc_info(similarity_score=0.7788359831734193, index=6364, title='One Froggy Evening')
ranked_doc_info(similarity_score=0.7242336304598822, index=9043, title='Golden Needles')
ranked_doc_info(similarity_score=0.7242336304598822, index=23040, title='Golden Needles')
ranked_doc_info(similarity_score=0.7181273499299481, index=4917, title='Incident')
ranked_doc_info(similarity_score=0.7093090308663202, index=18168, title='Men Like These')
ranked_doc_info(similarity_score=0.6985558194210205, index=1799, title='This Side of Heaven')
ranked_doc_info(similarity_score=0.6970466915975826, index=17711, title='The Tracker')
ranked_doc_info(similarity_score=0.6834659983178661, index=2170, title='Arizona Days')
ranked_doc_info(similarity_score=0.6754434904814736, index=18579, title='Facing the Music')


In [ ]:
query, title = (df["Plot"][1], df["Title"][1])
print(f"Query title is {title}")
_ = movie_LSI.top_k_retrieval(query, 10)

Query title is Love by the Light of the Moon
ranked_doc_info(similarity_score=1.0, index=1, title='Love by the Light of the Moon')
ranked_doc_info(similarity_score=0.7103499025885032, index=19385, title='The Golden Disc')
ranked_doc_info(similarity_score=0.6962069812869465, index=1627, title='The Affairs of Cellini')
ranked_doc_info(similarity_score=0.6920774924496321, index=24245, title='Brahmachari')
ranked_doc_info(similarity_score=0.6806928273896731, index=34528, title="You're My Pet")
ranked_doc_info(similarity_score=0.6466265563492684, index=14668, title='Her Minor Thing')
ranked_doc_info(similarity_score=0.6302469722156732, index=649, title='The Love of Sunya')
ranked_doc_info(similarity_score=0.6267921635420188, index=7134, title="The Thing That Couldn't Die")
ranked_doc_info(similarity_score=0.6261997046807164, index=23866, title='Everyday I Love You')
ranked_doc_info(similarity_score=0.61517784226755, index=19316, title='Face in the Night')


# Evaluation of the system

To evaluate the system, we must fix a set of queries and their results. To do this, we simply pick 100 random titles and their plot descriptions and use these as queries. Naturally, we expect that when we input the exact plot description, we obtain the movie title as the best match.

Furthermore, by tuning the components of the truncated svd, we can also observe the interplay between efficiency and effectiveness, which we will study below.

In [ ]:
n_samples = 100
fixed_queries = df.sample(n = n_samples, random_state = 1234).iloc[:,[1,7]]
fixed_queries

,Title,Plot
19551,Gorgo,Captain Joe Ryan is salvaging for treasure off...
8596,Rio Lobo,"During the American Civil War, Col. Cord McNal..."
10315,American Flyers,Sports physician Marcus Sommers (Costner) pers...
31206,Jilla,Sakthi (Vijay) is the adopted son of a Madurai...
1574,Parachute Jumper,United States Marine Corps Lieutenants and pil...
...,...,...
1228,Too Many Cooks,Engaged couple Albert Bennett (Bert Wheeler) a...
5097,The Desert Hawk,An arranged marriage forces Arabian Princess S...
20554,The Pope Must Die,The plot is predicated on the Vatican being co...
26959,Ya Rab,Ikram (Raju Kher) is a practicing Muslim livin...


Since we are only considering one relevant document per query, we will use the R-precision metric with R=1 and average over all the queries.
This can be done for different values of the components.

In [ ]:
import plotly.express as px
component_range = range(1,20,2)
R_precisions = []
movie_LSI = LSI(components = 1)
for k in component_range:
    movie_LSI.change_number_of_components(k)
    score = 0
    for index, row in fixed_queries.iterrows():
        retrieved_doc = movie_LSI.top_k_retrieval(row["Plot"], 1, print_info = False)
        if row["Title"] == retrieved_doc[0].title:
            score += 1
    score /= len(fixed_queries)
    R_precisions.append(score)
px.line(x = component_range, y = R_precisions)

According to the plot above, already with 2 components the R-precision is 0.98. This means that the LSI system returns the correct document as the first choice 98% of the time.

However, the 1-precision is not a realiable metric for evaluating a system which is of interest when combined with a top-k retrieval. 

Thus, we can perform a manual evaluation of the system by performing some queries and evaluating whether the top-5 results are relevant or not.

In [ ]:
query1 = "love and passion"
query2 = "unsolved mystery"
query3 = "violent crime"
query4 = "space and spaceships"
query5 = "robot attack"

k = 5

# System with 10 components
movie_LSI.change_number_of_components(components = 10)

############################################################################################
movies = movie_LSI.top_k_retrieval(query1, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision: 5/5

############################################################################################
movies = movie_LSI.top_k_retrieval(query2, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision 4/5

############################################################################################
movies = movie_LSI.top_k_retrieval(query3, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision 5/5

############################################################################################
movies = movie_LSI.top_k_retrieval(query4, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision: 5/5

############################################################################################
movies = movie_LSI.top_k_retrieval(query5, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision: 0/5


############################################################################################

# Average Precision: 19/25 = 76%

ranked_doc_info(similarity_score=0.9941961067211976, index=8699, title='Minnie and Moskowitz')
ranked_doc_info(similarity_score=0.9929493146586679, index=5682, title='Monsoon')
ranked_doc_info(similarity_score=0.9923776216001493, index=18286, title='The King of Paris')
ranked_doc_info(similarity_score=0.9907890693475677, index=34322, title='Deaf Sam-yong')
ranked_doc_info(similarity_score=0.9904225529365077, index=30687, title='Azhagai Irukkirai Bayamai Irukkirathu')


Movie name: Minnie and Moskowitz 
 	 Plot: Following a break-up, Minnie Moore, a museum curator, becomes disillusioned by love and meaningful relationships. But after a chance encounter, she meets Seymour Moskowitz, a parking-lot attendant. After this event, Moskowitz falls in love with Minnie, trying desperately to get her to love him back.
Movie name: Monsoon 
 	 Plot: A well established gentleman falls in love with his fiancé's sister.[original research?]
Movie name: The King of Paris 
 	 Plot: An influential actor an

In [ ]:
# System with 50 components
movie_LSI.change_number_of_components(components = 50)

############################################################################################
movies = movie_LSI.top_k_retrieval(query1, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision: 5/5

############################################################################################
movies = movie_LSI.top_k_retrieval(query2, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision 5/5

############################################################################################
movies = movie_LSI.top_k_retrieval(query3, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision 4/5

############################################################################################
movies = movie_LSI.top_k_retrieval(query4, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision: 5/5

############################################################################################
movies = movie_LSI.top_k_retrieval(query5, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision: 0/5


############################################################################################

# Average Precision: 19/25 = 76%

ranked_doc_info(similarity_score=0.9879231663884593, index=8699, title='Minnie and Moskowitz')
ranked_doc_info(similarity_score=0.9778991150397358, index=34322, title='Deaf Sam-yong')
ranked_doc_info(similarity_score=0.9750910192066355, index=5682, title='Monsoon')
ranked_doc_info(similarity_score=0.9607529652935641, index=31131, title='18 Vayasu')
ranked_doc_info(similarity_score=0.9547464046638545, index=18281, title='Give Her a Ring')


Movie name: Minnie and Moskowitz 
 	 Plot: Following a break-up, Minnie Moore, a museum curator, becomes disillusioned by love and meaningful relationships. But after a chance encounter, she meets Seymour Moskowitz, a parking-lot attendant. After this event, Moskowitz falls in love with Minnie, trying desperately to get her to love him back.
Movie name: Deaf Sam-yong 
 	 Plot: A deaf farmhand is in love with the landlord's daughter-in-law.
Movie name: Monsoon 
 	 Plot: A well established gentleman falls in love with his fiancé's sister.[original rese

In [ ]:
# System with 100 components
movie_LSI.change_number_of_components(components = 100)

############################################################################################
movies = movie_LSI.top_k_retrieval(query1, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision: 5/5

############################################################################################
movies = movie_LSI.top_k_retrieval(query2, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision 5/5

############################################################################################
movies = movie_LSI.top_k_retrieval(query3, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision 5/5

############################################################################################
movies = movie_LSI.top_k_retrieval(query4, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision: 5/5

############################################################################################
movies = movie_LSI.top_k_retrieval(query5, k)

print("\n")
for movie in movies:
    print(f"Movie name: {movie.title} \n \t Plot: {movie_LSI.get_from_corpus(movie.index)}")
print("\n\n")
# precision: 0/5


############################################################################################

# Average Precision: 20/25 = 80%

ranked_doc_info(similarity_score=0.9727272302198755, index=34322, title='Deaf Sam-yong')
ranked_doc_info(similarity_score=0.9617374550832621, index=5682, title='Monsoon')
ranked_doc_info(similarity_score=0.9579230813952669, index=8699, title='Minnie and Moskowitz')
ranked_doc_info(similarity_score=0.9356731921394733, index=30687, title='Azhagai Irukkirai Bayamai Irukkirathu')
ranked_doc_info(similarity_score=0.9190392227597822, index=31131, title='18 Vayasu')


Movie name: Deaf Sam-yong 
 	 Plot: A deaf farmhand is in love with the landlord's daughter-in-law.
Movie name: Monsoon 
 	 Plot: A well established gentleman falls in love with his fiancé's sister.[original research?]
Movie name: Minnie and Moskowitz 
 	 Plot: Following a break-up, Minnie Moore, a museum curator, becomes disillusioned by love and meaningful relationships. But after a chance encounter, she meets Seymour Moskowitz, a parking-lot attendant. After this event, Moskowitz falls in love with Minnie, trying desperately 

The results are:

- 10 components: 76% average precision
- 50 components: 76% average precision
- 100 components: 80% average precision

However, it is important to observe that these results are extremely dependent on the query and have no real statistical significance. In fact, we can see that the query "robot attacks" yields always non relevant results. This could be explained by the fact that perhaps the data set is imbalanced and contains predominantely another genre of movies. 

Another query that fails to yield results is the phrase "nature walks". This reveals that the evaluation of the system heavily depends on the dataset and thus one should spend more time studying the data set, classifying some documents as relevant and not relevant, and performing more queries. 

From our conclusions, it is impossible to decide how many components to keep, thus we assume that 100 components is a reasonable choice.


Lastly, we must keep an eye out for the baselines.

In LSI, the baseline is given by the time it takes to answer a query without doing any truncation and using the full term-document matrix. In fact, in this scenario, we are not loosing any information since the whole matrix is utilized. Therefore, any solution we may use, must be at least as fast as this solution; otherwise it is not worth it.

In [ ]:
movie_LSI = LSI(components = 100) #just to initialize and create the vectorizer
doc_term_matrix = movie_LSI.term_document_matrix.toarray()


In [ ]:
import time
tic = time.perf_counter()
for index, row in fixed_queries.iterrows():
    vectorized_text = movie_LSI.clean_and_vectorize_text(row["Plot"])
    norm_v = sparse.linalg.norm(vectorized_text)
    vectorized_text /= norm_v
    ranked_docs = []
    for i, row in enumerate(doc_term_matrix.T):
        row = row.reshape(2105,1)
        norm_r = norm(row)
        if norm_r < 1e-15:
            continue
        else:
            ranked_docs.append(ranked_doc_info(sparse.csr_matrix.dot(vectorized_text.T,row)/norm_r, i, titles[i]))
    ranked_docs.sort(key = lambda x:x[0], reverse = True)
toc = time.perf_counter()
result = (toc-tic)/len(fixed_queries)
print(f"The average time per query is {result} seconds")

The average time per query is 3.3564295248400047 seconds


Therefore, the LSI system must perform faster than this on average, for the IR system to be useful.

With 100 components, the system on average takes:

In [ ]:
tic = time.perf_counter()
for index, row in fixed_queries.iterrows():
    #note that for consistency with the timing above, we call the answer query method instead of top_k_retrieval
    #which only returns a sliced portion of the sorted array 
    retrieved_doc = movie_LSI.answer_query(row["Plot"])
toc = time.perf_counter()
result = (toc-tic)/len(fixed_queries)
print(f"The average time per query is {result} seconds")

The average time per query is 0.48215696265999214 seconds


The LSI system with 100 components is around 5x faster which is a good improvement. 

# Documents Clustering

One of the main applications of LSI is clustering of documents to gain a better understanding the distribution. To do this, we must first try to understand what each concept represents: this can be done through the matrix U. Since the matrix U is the term concept matrix, we can pick, for each concept, the 5 most important terms and choose these as representatives. 

This will give us an insight into the meaning of the concepts.

In [ ]:
components = 5
movie_LSI.change_number_of_components(components)
df = pd.DataFrame(movie_LSI.U, index = movie_LSI.vectorizer.get_feature_names_out())
for col in df:
    mask = df[col].nlargest(5).index
    print(df[col][mask])

police    0.098149
school    0.078877
car       0.066302
money     0.062540
murder    0.059384
Name: 0, dtype: float64
tom       0.263090
police    0.205570
jerry     0.131819
house     0.123038
money     0.119069
Name: 1, dtype: float64
kill       0.326924
police     0.225835
village    0.199161
brother    0.146907
murder     0.145160
Name: 2, dtype: float64
kill      0.175778
police    0.101807
escape    0.099800
shoot     0.096435
attack    0.087721
Name: 3, dtype: float64
toss       -0.003686
mortally   -0.003758
leap       -0.003787
horrify    -0.003792
yell       -0.003796
Name: 4, dtype: float64


We can see that all concepts seem to be geared towards horror-crime topics. This is result is completely dependent on the data so there is not much that can be done to improve these results. This is in contrast with other approaches such as LDA which are better at separating topics.

Lastly, to each document we can associate the most important concept and this can allow us to perform a clustering. 
We can plot all documents in a 2D axis of the first 2 documents and assign a color for each of the documents depending on the cluster they belong to.

In [ ]:
df = pd.DataFrame(movie_LSI.Vh)
clst_id = []
for col in df:
    clst_id.append(df[col].nlargest(1).index[0])


In [ ]:
px.scatter(x = df.iloc[0,:], y = df.iloc[1,:], color = clst_id)

# Improvements

Some possible improvements to this work could be to utilize PCA instead of SVD since in this way we can isolate the directions of higher variance/information therefore improving the quality of retrieval. Furthermore, it could be interesting to perhaps see if applications of PPCA or kernel PCA could be applied to these systems.

Lastly, it could be useful to compare to a method such as a word vector model.

#Conclusion

LSI is a IR system which improves the word vector model by handling problems such as synonymy and polysemy by representing documents and terms in a latent space. 

The system designed in this notebook shows the basic ideas and we notice that it is effective although hard to evaluate on the given data set.

The drawback of LSI is that the computation of the SVD is expensive and become problematic for corpuses of very high volume. However, for small corpi, it is a very versatile IR system. 